In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

eng = ['hello', 'good morning']
ger = ['hallo', 'guten Morgen']

eng_tok = Tokenizer()
ger_tok = Tokenizer()
eng_tok.fit_on_texts(eng)
ger_tok.fit_on_texts(ger)

X = eng_tok.texts_to_sequences(eng)
y = ger_tok.texts_to_sequences(ger)

X = pad_sequences(X, maxlen=1, padding='post')
y = pad_sequences(y, maxlen=1, padding='post')

X = np.expand_dims(X, -1)

enc_in = Input(shape=(1, 1))
enc_emb = Dense(8)(enc_in)
enc_out, state_h, state_c = LSTM(16, return_state=True)(enc_emb)
enc_states = [state_h, state_c]

dec_in = Input(shape=(1, 1))
dec_emb = Dense(8)(dec_in)
dec_lstm, _, _ = LSTM(16, return_sequences=True, return_state=True)(dec_emb, initial_state=enc_states)
dec_out = Dense(len(ger_tok.word_index)+1, activation='softmax')(dec_lstm)

model = Model([enc_in, dec_in], dec_out)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit([X, X], y, epochs=100, verbose=1)
print(model.summary())

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 1.5311 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 9ms/step - loss: 1.5148 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 1.4989 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 1.4831 - accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 1.4677 - accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 0s/step - loss: 1.4524 - accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 8ms/step - loss: 1.4375 - accuracy: 0.0000e+00
Epoch 8/100
1/1 [==============================] - 0s 11ms/step - loss: 1.4227 - accuracy: 0.0000e+00
Epoch 9/100
1/1 [==============================] - 0s 0s/step - loss: 1.4082 - accuracy: 0.0000e+00
Epoch 10/100
1/1 [==============================] - 0s 5ms/step - loss: 1.3940 - accuracy: 0

1/1 [==============================] - 0s 0s/step - loss: 0.6823 - accuracy: 0.5000
Epoch 85/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6748 - accuracy: 0.5000
Epoch 86/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6673 - accuracy: 0.5000
Epoch 87/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6598 - accuracy: 1.0000
Epoch 88/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6522 - accuracy: 1.0000
Epoch 89/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6447 - accuracy: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6372 - accuracy: 1.0000
Epoch 91/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6297 - accuracy: 1.0000
Epoch 92/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6222 - accuracy: 1.0000
Epoch 93/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6147 - accuracy: 1.0000
Epoch 94/100
1/1 [============

In [2]:
import numpy as np

def predict_translation(input_text):
    seq = eng_tok.texts_to_sequences([input_text])
    seq = pad_sequences(seq, maxlen=1, padding='post')
    seq = np.expand_dims(seq, -1)
    pred = model.predict([seq, seq])
    pred_idx = np.argmax(pred[0, 0])
    if pred_idx == 0:
        return ''
    return ger_tok.index_word[pred_idx]

print(predict_translation('I am'))
print(predict_translation('good morning'))


1/1 [==============================] - 1s 553ms/step
hallo
1/1 [==============================] - 0s 17ms/step
morgen
